Tested to be running in Colab (2024.03.23) On T4 GPU, without bf16 or tf32

Author: https://github.com/TongmengXie/

# pre-run

In [ ]:
# get ollama
!sudo apt-get install -y pciutils
!curl https://ollama.ai/install.sh | sh
import pandas as pd

# dependencies for fine-tuning
!pip install accelerate -U -q
!pip install deepspeed -q
!pip install flash-attn --no-build-isolation -q # flash attenion and for vision-tower

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 7.4 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.5.6-cp310-cp310-linux_x86_64.whl size=120592258 sha256=d8cf54adda65f59820221d329d274e124972d7fdc05ab3b1130253c64eee6c8a
  Stored in directory: /root/.cache/pip/wheels/a8/1c/88/b959d6818b98a46d61ba231683abb7523b89ac1a7ed1e0c206
Successfully built flash-attn


In [ ]:
# clone from myself
!git clone https://github.com/Tongmengxie/LLaVA.git

# optional: clone from public repo after my pull request merged
# %cd LLaVA
# !git clone https://github.com/haotian-liu/LLaVA.git
# !git checkout colab
# %cd ..
# %pwd
!mkdir LLaVA/checkpoints/llava-v1.5-13b-task-lora

# Input & Output Directories

In [ ]:
# data_path
# DATA_PATH = 'LLaVA/playground/data/llava_v1_5_mix665k.json'
DATA_PATH = 'path_to_data' # see for format: https://github.com/haotian-liu/LLaVA/blob/main/docs/Finetune_Custom_Data.md

# image_folder
# IMAGE_FOLDER = 'LLaVA/playground/data'
IMAGE_FOLDER = 'path_to_images'

In [ ]:
!deepspeed LLaVA/llava/train/train_mem.py \
    --lora_enable True --lora_r 128 --lora_alpha 256 --mm_projector_lr 2e-5 \
    --deepspeed LLaVA/scripts/zero3.json \
    --model_name_or_path liuhaotian/llava-v1.5-13b \
    --version v1 \
    --data_path DATA_PATH \
    --image_folder IMAGE_FOLDER \
    --mm_projector_type mlp2x_gelu \
    --mm_vision_select_layer -2 \
    --mm_use_im_start_end False \
    --mm_use_im_patch_token False \
    --image_aspect_ratio pad \
    --group_by_modality_length True \
    --bf16 False \
    --output_dir LLaVA/checkpoints/llava-v1.5-13b-task-lora \
    --num_train_epochs 1 \
    --per_device_train_batch_size 16 \
    --per_device_eval_batch_size 4 \
    --gradient_accumulation_steps 1 \
    --evaluation_strategy "no" \
    --save_strategy "steps" \
    --save_steps 50000 \
    --save_total_limit 1 \
    --learning_rate 2e-4 \
    --weight_decay 0. \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --tf32 False \
    --model_max_length 2048 \
    --gradient_checkpointing True \
    --dataloader_num_workers 4 \
    --lazy_preprocess True \
    --report_to wandb
    # --vision_tower openai/clip-vit-large-patch14-336 \

[2024-03-23 17:49:06,369] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-03-23 17:49:10,197] [WARNING] [runner.py:202:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2024-03-23 17:49:10,211] [INFO] [runner.py:568:main] cmd = /usr/bin/python3 -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 --enable_each_rank_log=None LLaVA/llava/train/train_mem.py --lora_enable True --lora_r 128 --lora_alpha 256 --mm_projector_lr 2e-5 --deepspeed LLaVA/scripts/zero3.json --model_name_or_path liuhaotian/llava-v1.5-13b --version v1 --data_path LLaVA/playground/data/llava_v1_5_mix665k.json --image_folder LLaVA/playground/data --mm_projector_type mlp2x_gelu --mm_vision_select_layer -2 --mm_use_im_start_end False --mm_use_im_patch_token False --image_aspect_ratio pad --group_by_modality_length True --bf16 False --output_dir LLaVA/checkpoints/ll